In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader

# Load the dataset

In [2]:
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv", index_col=0)

In [3]:
train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
id,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


# Process the dataset to remove null

In [4]:
train["prompt"] = train.prompt.map(lambda x: eval(x)[0])
train["response_a"] = train.response_a.map(lambda x: eval(x.replace("null","''"))[0])
train["response_b"] = train.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Generate categorical data

In [5]:
train["class_name"] = train[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
train["class_label"] = train.class_name.map({'winner_model_a':0, 'winner_model_b': 1, 'winner_tie' : 2})


train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label
id,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1
65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,winner_tie,2
96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,winner_model_a,0
198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,winner_model_b,1


# Make pair by joining the prompt and response

In [6]:
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except Exception:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except Exception:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except Exception:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",
                      f"Prompt: {prompt}\n\nResponse: {response_b}"
                     ]
    return row

In [7]:
train = train.apply(make_pairs, axis=1)  
train.head(2)

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label,encode_fail,options
id,,,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0,False,[Prompt: Is it morally right to try to have a ...
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1,False,[Prompt: What is the difference between marria...


# Drop failed UTF-8 checking

In [8]:
print(train.encode_fail.value_counts(normalize=False))
train = train[train['encode_fail'] == False]
print(train.encode_fail.value_counts(normalize=False))


encode_fail
False    56885
True       592
Name: count, dtype: int64
encode_fail
False    56885
Name: count, dtype: int64


# Remove unwanted columns

In [9]:
from sklearn.model_selection import train_test_split  # Import package

train = train.drop(['model_a', 'model_b', 'prompt', 'response_a','response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'class_name'], axis=1)

train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train["class_label"])

In [10]:
train_df.shape

(45508, 3)

# Make match dataloader

In [11]:
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):
    def __init__(self, dataSet: pd.DataFrame, tokenizer):
        self.labels = dataSet['class_label'].values
        self.values = dataSet['options'].values
        self.dataSet = dataSet
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        text = self.values[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = torch.reshape(encoding['input_ids'], (-1, ))
        attention_mask = torch.reshape(encoding['attention_mask'], (-1, ))
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load Models for tokeniser and classification from Hugging face

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
model

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2025-06-02 06:13:01.496590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748844781.700554      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748844781.758084      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [13]:
train_dataSet = TextClassificationDataset(train_df, tokenizer)
valid_dataSet = TextClassificationDataset(valid_df, tokenizer)

batch_size = 75
train_loader = DataLoader(train_dataSet, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataSet, batch_size=batch_size)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [15]:
!nvidia-smi

Mon Jun  2 06:13:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             32W /  250W |     549MiB /  16384MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), 2e-5)

# Training and eval loop

In [17]:
from tqdm import tqdm

num_epochs = 5
learning_rate = 2e-5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    epoch_loss = 0
    model.train()
    train_pbar = tqdm(train_loader, desc="Training", leave=False)
    
    for batch in train_pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Average Training Loss: {avg_epoch_loss:.4f}")
    
    valid_err = 0
    model.eval()
    valid_pbar = tqdm(valid_loader, desc="Validation", leave=False)
    
    with torch.no_grad():
        for batch in valid_pbar:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            valid_err += loss.item()
            
            valid_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_valid_loss = valid_err / len(valid_loader)
    print(f"Average Validation Loss: {avg_valid_loss:.4f}")


Epoch 1/5


Average Training Loss: 1.0731


Average Validation Loss: 1.0693

Epoch 2/5


Average Training Loss: 1.0286


Average Validation Loss: 1.0466

Epoch 3/5


...

Average Validation Loss: 1.0604

Epoch 4/5


Average Training Loss: 0.8590


Average Validation Loss: 1.1602

Epoch 5/5


Average Training Loss: 0.7051


Average Validation Loss: 1.2736


# Save the model

In [18]:
torch.save(model.state_dict(), "model.pt")
torch.save(model, "model.h5")